In [1]:
!pip install rasterio

  Using cached cligj-0.7.2-py3-none-any.whl.metadata (5.0 kB)
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl.metadata (6.4 kB)
   ---------------------------------------- 0.0/24.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.8 MB 1.3 MB/s eta 0:00:20
   ---------------------------------------- 0.2/24.8 MB 2.3 MB/s eta 0:00:11
    --------------------------------------- 0.3/24.8 MB 3.1 MB/s eta 0:00:08
    --------------------------------------- 0.5/24.8 MB 3.0 MB/s eta 0:00:09
   - -------------------------------------- 0.7/24.8 MB 3.3 MB/s eta 0:00:08
   - -------------------------------------- 0.9/24.8 MB 3.5 MB/s eta 0:00:07
   - -------------------------------------- 1.1/24.8 MB 3.8 MB/s eta 0:00:07
   -- ------------------------------------- 1.4/24.8 MB 4.1 MB/s eta 0:00:06
   -- ------------------------------------- 1.7/24.8 MB 4.3 MB/s eta 0:00:06
   --- ------------------------------------ 2.0/24.8 MB 4.5 MB/s eta 0:00:06
   --- -----------------


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: C:\Users\owd1\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import os
import rasterio
import numpy as np
from rasterio.windows import Window
from rasterio.enums import Resampling
from rasterio.transform import from_origin
from tqdm import tqdm

# Define the path to the downloaded image
input_image_path = r"H:\My Drive\Riyadh_folder (1)\Riyadh_Full_ROI.tif"

# Define the output directory
output_directory = r"H:\My Drive\r"
os.makedirs(output_directory, exist_ok=True)

# Open the image
with rasterio.open(input_image_path) as src:
    # Read the QA60 band (assuming it's the 5th band)
    qa60_band_index = 5  # 1-based indexing in rasterio
    # Get image dimensions
    width = src.width
    height = src.height
    bands = src.count
    
    # Calculate the number of segments in x and y directions
    tile_size = 256
    x_tiles = (width + tile_size - 1) // tile_size
    y_tiles = (height + tile_size - 1) // tile_size

    # Statistics counters
    total_tiles = x_tiles * y_tiles
    passed_tiles = 0
    dropped_tiles = 0

    print(f"Total tiles to process: {total_tiles}")

    # Loop over each tile
    for y in tqdm(range(y_tiles), desc='Processing tiles'):
        for x in range(x_tiles):
            # Define window
            window = Window(
                col_off=x * tile_size,
                row_off=y * tile_size,
                width=min(tile_size, width - x * tile_size),
                height=min(tile_size, height - y * tile_size)
            )
            
            # Read QA60 band for the current window
            qa60 = src.read(
                qa60_band_index,
                window=window,
                out_shape=(window.height, window.width),
                resampling=Resampling.nearest
            )
            
            # Create cloud mask
            qa60 = qa60.astype(np.uint16)
            cloud_mask = ((qa60 & (1 << 10)) == 0) & ((qa60 & (1 << 11)) == 0)
            
            # Check if the tile is cloud-free (e.g., less than 10% cloud cover)
            cloud_cover = np.mean(~cloud_mask)
            cloud_threshold = 0.05  # 10% cloud cover threshold
            
            if cloud_cover <= cloud_threshold:
                # Tile is acceptable, read all bands
                data = src.read(
                    indexes=[1, 2, 3, 4],  # Bands B8, B4, B3, B2
                    window=window,
                    out_shape=(4, window.height, window.width),
                    resampling=Resampling.nearest
                )
                
                # Apply cloud mask to the data
                data = np.where(cloud_mask, data, np.nan)
                
                # Prepare metadata for the output file
                transform = src.window_transform(window)
                out_meta = src.meta.copy()
                out_meta.update({
                    "driver": "GTiff",
                    "height": window.height,
                    "width": window.width,
                    "transform": transform,
                    "count": 4  # Number of bands
                })
                
                # Save the tile
                tile_filename = f"tile_y{y}_x{x}.tif"
                tile_path = os.path.join(output_directory, tile_filename)
                
                with rasterio.open(tile_path, 'w', **out_meta) as dest:
                    dest.write(data)
                
                passed_tiles += 1
            else:
                # Tile is discarded due to cloud cover
                dropped_tiles += 1

    print(f"Total tiles processed: {total_tiles}")
    print(f"Tiles passed (cloud-free): {passed_tiles}")
    print(f"Tiles dropped (cloudy): {dropped_tiles}")


Total tiles to process: 576


Processing tiles: 100%|██████████| 24/24 [00:28<00:00,  1.18s/it]

Total tiles processed: 576
Tiles passed (cloud-free): 576
Tiles dropped (cloudy): 0
